<a href="https://colab.research.google.com/github/fidan-c/human-value-detection/blob/main/Value_Det2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import random
import numpy as np


In [ ]:
random.seed(64)
np.random.seed(64)

In [ ]:
# using Conc + Premise as one representation and Stance added to the labels: roberta-large. No token added when concatenating the two columns to one column.
!pip install datasets evaluate transformers[sentencepiece]
!pip install transformers
!pip install tokenizer
!pip install simpletransformers

In [ ]:

#from transformers import AutoTokenizer,TFAutoModel, DataCollatorWithPadding
import numpy as np
import pandas as pd
#from datasets import load_dataset  # was necessary for using Hugging Face transformers to have a dictonary, not needed for "simple transformers".

In [ ]:
from simpletransformers.classification import (
    MultiLabelClassificationModel, MultiLabelClassificationArgs
)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# load data - not including chinese
# sep for tsv
arguments_training_no_label = pd.read_csv('/content/drive/MyDrive/transformers/arguments-training.tsv', sep='\t') 
arguments_validation_no_label = pd.read_csv('/content/drive/MyDrive/transformers/arguments-validation.tsv', sep='\t')
arguments_test_data = pd.read_table('/content/drive/MyDrive/transformers/arguments-test.tsv')
arguments_test_data_nahjalbalagha= pd.read_table('/content/drive/MyDrive/transformers/arguments-test-nahjalbalagha.tsv')

labels_training = pd.read_csv('/content/drive/MyDrive/transformers/labels-training.tsv', sep='\t').drop('Argument ID', axis=1)
labels_validation= pd.read_csv('/content/drive/MyDrive/transformers/labels-validation.tsv', sep='\t').drop('Argument ID', axis=1)

print(labels_training.shape)
labels_training.head()


#new added label

label_to_add = ["Stance"]

# fixing the dataset for inconsistency: in favor of, in favour of
arguments_training_no_label['Stance'].replace(['in favour of'],
                       ['in favor of'], inplace=True)

arguments_validation_no_label['Stance'].replace(['in favour of'],
                       ['in favor of'], inplace=True)


arguments_test_data['Stance'].replace(['in favour of'],
                       ['in favor of'], inplace=True)

arguments_test_data_nahjalbalagha['Stance'].replace(['in favour of'],
                       ['in favor of'], inplace=True)

# from categorical value to numerical value for Stance

arguments_training_no_label['Stance'].replace(['in favor of', 'against'],
                       [1, 0], inplace=True)

arguments_validation_no_label['Stance'].replace(['in favor of', 'against'],
                       [1, 0], inplace=True)


arguments_test_data['Stance'].replace(['in favor of', 'against'],
                       [1, 0], inplace=True)

arguments_test_data_nahjalbalagha['Stance'].replace(['in favor of','against'],
                                                    [1,0], inplace=True)







(5393, 20)


In [ ]:
arguments_validation_no_label.head()


,Argument ID,Conclusion,Stance,Premise
0,A01001,Entrapment should be legalized,1,if entrapment can serve to more easily capture...
1,A01012,The use of public defenders should be mandatory,1,the use of public defenders should be mandator...
2,A02001,Payday loans should be banned,1,payday loans create a more impoverished societ...
3,A02002,Surrogacy should be banned,0,Surrogacy should not be banned as it is the wo...
4,A02009,Entrapment should be legalized,0,entrapment is gravely immoral and against huma...


In [ ]:
labels_training['Stance_As_Label'] = arguments_training_no_label[label_to_add] 
#labels_training = pd.concat([labels_training,arguments_training_no_label[label_to_add]], axis=1)
print(labels_training.shape)
print(labels_training.head())

labels_validation['Stance_As_Label'] = arguments_validation_no_label[label_to_add]
#labels_validation = pd.concat([labels_validation,arguments_validation_no_label[label_to_add]], axis=1)



print(labels_validation.shape)
print(labels_validation.head())


(5393, 21)
   Self-direction: thought  Self-direction: action  Stimulation  Hedonism  \
0                        0                       0            0         0   
1                        0                       0            0         0   
2                        0                       0            0         0   
3                        0                       0            0         0   
4                        0                       0            0         0   

   Achievement  Power: dominance  Power: resources  Face  Security: personal  \
0            0                 0                 0     0                   0   
1            0                 0                 0     0                   1   
2            0                 1                 0     0                   0   
3            0                 0                 0     0                   0   
4            0                 0                 0     0                   1   

   Security: societal  ...  Conformity: rules

In [ ]:
# train_df = pd.concat([arguments_training_no_label, labels_training], axis=1)

# val_df= pd.concat([arguments_validation_no_label, labels_validation], axis=1)

train_labels = labels_training.values.tolist()

#concat_input = pd.concat([arguments_training_no_label['Conclusion'],arguments_training_no_label['Stance'],arguments_training_no_label['Premise']], axis=1)

cols_to_concat = ["Conclusion", "Premise"]

arguments_training_no_label['Concated'] = arguments_training_no_label[cols_to_concat].apply(lambda x: ' '.join(x.astype(str)),
    axis=1)

print(arguments_training_no_label['Concated'][0])

#lower every word - roberta is case sensitive, thats why
arguments_training_no_label['Concated']= arguments_training_no_label['Concated'].str.lower()

print(arguments_training_no_label['Concated'][0])

train_text = arguments_training_no_label['Concated'].values
train_data = [[t, l] for t, l in zip(train_text, train_labels)]  # see the format needed ["text",[label]]
train_df = pd.DataFrame(train_data)
train_df.columns = ["text", "labels"]


We should ban human cloning we should ban human cloning as it will only cause huge issues when you have a bunch of the same humans running around all acting the same.
we should ban human cloning we should ban human cloning as it will only cause huge issues when you have a bunch of the same humans running around all acting the same.


In [ ]:
type(train_labels)
#print(train_labels[0:3])
train_df.head()
train_df.shape

(5393, 2)

In [ ]:
train_text[0]


'we should ban human cloning we should ban human cloning as it will only cause huge issues when you have a bunch of the same humans running around all acting the same.'

In [ ]:
print(train_df.head())
print(train_labels[0:2])

                                                text  \
0  we should ban human cloning we should ban huma...   
1  we should ban fast food fast food should be ba...   
2  we should end the use of economic sanctions so...   
3  we should abolish capital punishment capital p...   
4  we should ban factory farming factory farming ...   

                                              labels  
0  [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ...  
1  [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ...  
2  [0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, ...  
3  [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, ...  
4  [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, ...  
[[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1], [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]]


In [ ]:

# repeat above for the validation set
val_labels = labels_validation.values.tolist()

arguments_validation_no_label['Concated'] = arguments_validation_no_label[cols_to_concat].apply(lambda x: ' '.join(x.astype(str)),
    axis=1)

print(arguments_validation_no_label['Concated'][0])
arguments_validation_no_label['Concated']= arguments_validation_no_label['Concated'].str.lower()
print(arguments_validation_no_label['Concated'][0])


val_text = arguments_validation_no_label['Concated'].values
val_data = [[t, l] for t, l in zip(val_text, val_labels)] 
val_df = pd.DataFrame(val_data)
val_df.columns = ["text", "labels"]

Entrapment should be legalized if entrapment can serve to more easily capture wanted criminals, then why shouldn't it be legal?
entrapment should be legalized if entrapment can serve to more easily capture wanted criminals, then why shouldn't it be legal?


In [ ]:
# repeat above for the main test set

arguments_test_data['Concated'] = arguments_test_data[cols_to_concat].apply(lambda x: ' '.join(x.astype(str)),
    axis=1)

test_data = arguments_test_data['Concated'].values
np.shape(test_data)

print(arguments_test_data['Concated'][0])
arguments_test_data['Concated']= arguments_test_data['Concated'].str.lower()
print(arguments_test_data['Concated'][0])

We should end affirmative action affirmative action helps with employment equity.
we should end affirmative action affirmative action helps with employment equity.


In [ ]:
arguments_test_data_nahjalbalagha['Concated'] = arguments_test_data_nahjalbalagha[cols_to_concat].apply(lambda x: ' '.join(x.astype(str)),
    axis=1)

test_data_nahjalbalagha = arguments_test_data_nahjalbalagha['Concated'].values
print(np.shape(test_data_nahjalbalagha))
print(test_data_nahjalbalagha[0])

arguments_test_data_nahjalbalagha['Concated']= arguments_test_data_nahjalbalagha['Concated'].str.lower()
print(arguments_test_data_nahjalbalagha['Concated'][0])

(279,)
Habitual greed devalues humans Greed is like a slippery rock on which even the steps of scholars are not firm.
habitual greed devalues humans greed is like a slippery rock on which even the steps of scholars are not firm.


In [ ]:
train_df['labels'][0:3]
train_df.shape

(5393, 2)

In [ ]:
for i in range(len(train_df)):
  for j in range(len(train_df['labels'][0])):
    if type(train_df['labels'][i][j]) == str:
      print(i,j, train_df['labels'][i][j] == 'in favour of')

In [ ]:
model_args = MultiLabelClassificationArgs(num_train_epochs=5, train_batch_size=16, overwrite_output_dir=True)

# Create a MultiLabelClassificationModel from simple transformers: <https://simpletransformers.ai/docs/multi-label-classification/>
model = MultiLabelClassificationModel(
    "roberta",
    "roberta-large",
    num_labels=len(labels_training.columns),
    args=model_args,
)

# Train the model
model.train_model(train_df)

Some weights of the model checkpoint at roberta-large were not used when initializing RobertaForMultiLabelSequenceClassification: ['lm_head.decoder.weight', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.bias', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing RobertaForMultiLabelSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForMultiLabelSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForMultiLabelSequenceClassification were not initialized from the model checkpoint at roberta-large and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'cl

  0%|          | 0/5393 [00:00<?, ?it/s]

Epoch:   0%|          | 0/5 [00:00<?, ?it/s]

Running Epoch 0 of 5:   0%|          | 0/338 [00:00<?, ?it/s]

Running Epoch 1 of 5:   0%|          | 0/338 [00:00<?, ?it/s]

Running Epoch 2 of 5:   0%|          | 0/338 [00:00<?, ?it/s]

Running Epoch 3 of 5:   0%|          | 0/338 [00:00<?, ?it/s]

Running Epoch 4 of 5:   0%|          | 0/338 [00:00<?, ?it/s]

(1690, 0.26498439669520896)

In [ ]:
result, model_outputs, wrong_predictions = model.eval_model(val_df)

  0%|          | 0/1896 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/237 [00:00<?, ?it/s]

In [ ]:
result

{'LRAP': 0.7890364115191275, 'eval_loss': 0.30749734758324765}

In [ ]:
# preds for the main test
test_preds = model.predict(list(test_data))

  0%|          | 0/1576 [00:00<?, ?it/s]

  0%|          | 0/197 [00:00<?, ?it/s]

In [ ]:
# pred outputs for the main test
cols = ["Argument ID"] + list(labels_validation.columns)
pred_data = []

for arg_id, labels in zip(arguments_test_data["Argument ID"].values, test_preds[0]):
  row = [arg_id] + labels[:]
  pred_data.append(row)

pred_df = pd.DataFrame(pred_data, columns=cols)
pred_df_no_stance = pred_df.drop("Stance_As_Label",axis=1)

pred_df_no_stance.to_csv("main_test_pred.tsv", index=False, sep='\t')

In [ ]:
# predictions for the nahjalbalagha test set
test_preds_nahjalbalagha= model.predict(list(test_data_nahjalbalagha))

  0%|          | 0/279 [00:00<?, ?it/s]

  0%|          | 0/35 [00:00<?, ?it/s]

In [ ]:
# prediction outputs for the nahjalbalagha test set
cols = ["Argument ID"] + list(labels_validation.columns)
pred_data_nah = []

for arg_id, labels in zip(arguments_test_data_nahjalbalagha["Argument ID"].values, test_preds_nahjalbalagha[0]):
  row = [arg_id] + labels[:]
  pred_data_nah.append(row)

pred_nah_df = pd.DataFrame(pred_data_nah, columns=cols)
pred_df_nah_no_stance = pred_nah_df.drop("Stance_As_Label",axis=1)

pred_df_nah_no_stance.to_csv("nahjalbalagha_test_pred.tsv", index=False, sep='\t')

In [ ]:
! cp /content/outputs/pytorch_model.bin /content/drive/MyDrive/pytorch_model.bin